This Jupyter notebook computes certain quantities related to the rejection
sampling probabilities for the <span style="font-variant:small-caps;">
RejSamplingMod</span> and <span style="font-variant:small-caps;">
SIMD-RejSamplingMod</span> algorithms in our paper. The results obtained from
these calculations are used in Section 4 ("Implementation aspects"), under
"Choosing the parameter $L$".

To run the notebook, a [SageMath](https://www.sagemath.org) kernel is required.

In [ ]:
# We perform the calculation with 256 bits of floating-point precision to have
# a good margin of safety; computations take on the order of a minute on a
# modern computer, despite the use of multiprecision arithmetic. We have
# checked with even higher precisions to ensure the results have converged.
prec = 256
RRR = RealField(prec)
x = polygen(RRR, 'x')

In [ ]:
# Initialize the parameter L (as in our paper), the parameter W of
# SIMD-RejSamplingMod, standardized values of n from the NTRU specification,
# and the acceptable probability of error (the rationale for the default
# choice is given in our paper).
L = 16
W = 16
ns = [509, 677, 821]
log2_p_err = -74
p_err = RRR(2^log2_p_err)

We recall that <span style="font-variant:small-caps;">RejSamplingMod</span>
outputs an array $\mathbf{si}$ of $n - 1$ elements, such that $\mathbf{si}[i]
\sim \mathcal{U}(0, n - 1 - i)$. It does so by rejection sampling. According to
Lemma 1 in the paper, the rejection probability for each $i \in {0, \ldots,
n - 2}$ is given by $p_r(i) = \frac{2^L \bmod (n - 1 - i)}{2^L}$, and thus the
acceptance probability is $p_a(i) = 1 - p_r(i)$ -- this corresponds to the array
`p` in the next cell.

We seek, for each $i$, the expected number of trials required until acceptance.
This can be modeled as a geometric random variable $X_i$. The probability that
the $k$-th trial is the first success is given by $\mathrm{Pr}(X_i = k) =
(1 - p_a(i))^{k - 1} p_a(i)$, and is computed as the variable `t` in the next
cell.

We then construct a probability-generating function $G_i(x) =
\sum_{k = 1}^\infty \mathrm{Pr}(X_i = k) x^k$, although we truncate the series
at $k < k_0$, where $k_0$ is such that $\mathrm{Pr}(X_i = k_0) < 2^{-prec}$,
with $prec$ the floating-point precision defined before. While this choice is
heuristic, our experiments indicate it is more than adequate. This is computed
as the variable `p_X_eq_k` in the next cell, and we construct an array `PGFs`
containing $G_i(x)$ for each $i \in {0, \ldots, n - 2}$.

In [ ]:
def compute_PGFs(RRR, n, L):
    p = [RRR(1 - (2^L % (n - 1 - i)) / 2^L) for i in range(n - 1)]
    PGFs = []
    
    for i in range(len(p)):
        p_X_eq_k = 0
        k = 1
        while True:
            t = (1 - p[i])^(k - 1)*p[i]
            if t < RRR(2^(-RRR.precision())):
                break
            p_X_eq_k += t*x^k
            k += 1
        PGFs.append(p_X_eq_k)
    
    return PGFs

Our algorithm <span style="font-variant:small-caps;">SIMD-RejSamplingMod</span>
performs rejection sampling of a block of $W$ entries of $\mathbf{si}$ at a
time. If a single entry within the block is rejected, we must run a fixup
procedure using scalar code which adds to the running time. It is imperative to
determine how often this happens. Thus, we seek the probability, for each block
$i = 0, \ldots, W - 1; W, \ldots, 2W - 1; \ldots$, that at least one of the
entries within the block is rejected.

We compute this using the result that the PGF of a sum of random variables
corresponds to the product of PGFs of the random variables. In the next cell, we
compute $\mathbf{resW}[j] = \sum_{i = jW}^{(j + 1)W - 1} X_i =
\prod_{i = jW}^{(j + 1)W - 1} G_i(x)$. Thus, $\mathbf{resW}[j]$ defines the PGF
for the number of trials in the $j$-th block, formed by indices $i = jW, \ldots,
(j + 1)W - 1$. The last range is treated separately as in general it may have
fewer than $W$ elements.

In [ ]:
def compute_resW(n, W, PGFs):
    nW = n - 1 - ((n - 1) % W)

    resW = [prod(PGFs[i:i + W]) for i in range(0, nW, W)] + [prod(PGFs[nW:])]

    return resW

Recall that the coefficient of $x^k$ in a PGF indicates the probability that $k$
trials are sufficient. We construct an array with the coefficients of $x^W$ for
each block (except for the last, in which we seek the coefficient of
$x^{(n - 1) \bmod W}$). We then take the minimum across this array; this is the
minimum probability, across all blocks, of full block acceptance, without the
need to run the fixup procedure.

By multiplying together all elements of the array `resW`, we obtain a PGF $X$
describing the global number of trials required for all $n - 1$ samples during a
full execution of <span style="font-variant:small-caps;">RejSamplingMod</span>.
The coefficient of $x^{n - 1}$ of this PGF is the probability that no rejections
at all happen during the execution of the algorithm -- not only for a given
block of $w$ entries, but across all blocks.

Finally, we determine the bound the number of uniform random integers required
in the $\mathbf{rnd}$ array, so that the probability of running out of random
integers due to excessive rejections is smaller than $p_{err}$. In other words,
we obtain $k$ such that $1 - \sum_{j = 0}^k \textrm{Pr}(X = n - 1 + k) <
p_{err}$. We also show the value of $k$ rounded up to a multiple of 8, a choice
tied to our use of NEON and $L = 16$, as explained in the paper.

In [ ]:
for n in ns:
    PGFs = compute_PGFs(RRR, n, L)
    resW = compute_resW(n, W, PGFs)

    pW = [resW[i][W] for i in range(len(resW) - 1)] + [resW[-1][(n - 1) % W]]

    print(
        f"Lowest probability of full block acceptance across all blocks for "
        f"n = {n}: {min(pW)}")

    res = prod(resW)

    print(
        f"Probability of no rejections in a full execution of the algorithm "
        f"for n = {n}: {res[n - 1]}")

    cum_p = RRR(0)
    for i in range(n - 1, res.degree() + 1):
        cum_p += res[i]
        if 1 - cum_p < p_err:
            print(
                f"Required length of rnd array of {L}-bit random integers to "
                f"achieve error probability < 2^{log2_p_err} for n = {n}: {i} "
                f"(rounded up to the next multiple of 8: {i + ((8 - i) % 8)})"
            )
            break
    
    print()